# Partial success !

We can now explode a small xml file into 8k+ rows.  
The single table contains 99% duplicated data, so we need to have multiple tables with foreign keys.

# install libs

In [110]:
!pip install flatten_dict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# declare working functions

In [111]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
from flatten_dict import flatten

def parse_xml(url):
    response = requests.get(url)
    data = response.text
    
    # Parse XML into ElementTree object
    root = ET.fromstring(data)
    
    # Convert ElementTree object into nested dictionary
    xml_dict = element_to_dict(root)
    
    # Flatten the nested dictionary
    flat_dict = flatten(xml_dict, reducer='underscore')
    
    df = pd.DataFrame.from_dict(flat_dict, orient='index').transpose()
    return df


def parse_xml_from_text(input_text):   
    # Flatten the nested dictionary
    flat_dict = flatten(input_text, reducer='underscore')
    
    df = pd.DataFrame.from_dict(flat_dict, orient='index').transpose()
    return df

# Helper function to convert ElementTree object to nested dictionary
def element_to_dict(element):
    result = {}
    
    # Store tag name as key
    result[element.tag] = {}
    
    # Store attributes as sub-dictionary if present
    if element.attrib:
        result[element.tag]['attributes'] = element.attrib
    
    # Store text content if present
    if element.text and element.text.strip():
        result[element.tag]['text'] = element.text.strip()
    
    # Process child elements recursively
    for child in element:
        child_dict = element_to_dict(child)
        if child.tag in result[element.tag]:
            # Handle multiple elements with the same tag name
            if isinstance(result[element.tag][child.tag], list):
                result[element.tag][child.tag].append(child_dict[child.tag])
            else:
                result[element.tag][child.tag] = [result[element.tag][child.tag], child_dict[child.tag]]
        else:
            result[element.tag].update(child_dict)
    
    return result

url = "https://raw.githubusercontent.com/louispaulet/gpt4_readline/main/simple%20text/complete_declaration.txt"
df = parse_xml(url)
df_long = df.T.reset_index()
df_long

,index,0
0,declaration_dateDepot_text,27/11/2022 18:18:23
1,declaration_uuid_text,fa8d18ec-0db9-4a39-b1f4-caba0c31329b
2,declaration_origine_text,ADEL
3,declaration_complete_text,true
4,declaration_attachedFiles_attachedFiles_fileNa...,VUE_PDF_DU_RECEPISSE_DU_DEPOT_XML
5,declaration_declarationVersion_text,20171221
6,declaration_activConsultantDto_neant_text,true
7,declaration_activProfCinqDerniereDto_items_ite...,CREATION
8,declaration_activProfCinqDerniereDto_items_ite...,Fonction exercée du 20 Mai au 4 Juillet 2022 \...
9,declaration_activProfCinqDerniereDto_items_ite...,"Ministre des Solidarités, de l'Autonomie et de..."


In [112]:
def explode_and_concat(df, column):
    # Create a temporary DataFrame that contains the column to be exploded along with the other columns in the original DataFrame
    temp_df = df.copy()
    
    # Explode the column containing lists
    temp_df = temp_df.explode(column)
    
    
    # Normalize the DataFrame
    df_to_normalize = pd.json_normalize(temp_df[column])
    df_to_normalize = df_to_normalize.add_prefix(column + '_')
    
    # Merge the original DataFrame with the normalized DataFrame
    result = pd.concat([temp_df.reset_index(drop=True), df_to_normalize.reset_index(drop=True)], axis=1)
    
    # Drop the original column as its content has been normalized and added as new columns
    result = result.drop(columns=[column])
    
    return result

# showing safe path

In [113]:
columns_to_explode = [
    'declaration_mandatElectifDto_items_items',
    'declaration_mandatElectifDto_items_items_remuneration.montant.montant',
    'declaration_participationDirigeantDto_items_items',
    'declaration_participationFinanciereDto_items_items',
    'declaration_activCollaborateursDto_items_items',
    'declaration_participationDirigeantDto_items_items_remuneration.montant.montant'
]

exploded_df = df

for column in columns_to_explode:
    exploded_df = explode_and_concat(exploded_df, column)


In [114]:
exploded_df

,declaration_dateDepot_text,declaration_uuid_text,declaration_origine_text,declaration_complete_text,declaration_attachedFiles_attachedFiles_fileName_text,declaration_declarationVersion_text,declaration_activConsultantDto_neant_text,declaration_activProfCinqDerniereDto_items_items_motif_id_text,declaration_activProfCinqDerniereDto_items_items_commentaire_text,declaration_activProfCinqDerniereDto_items_items_description_text,...,declaration_participationFinanciereDto_items_items_evaluation.text,declaration_participationFinanciereDto_items_items_remuneration.text,declaration_participationFinanciereDto_items_items_nombreParts.text,declaration_participationFinanciereDto_items_items_actiConseil.text,declaration_activCollaborateursDto_items_items_motif.id.text,declaration_activCollaborateursDto_items_items_nom.text,declaration_activCollaborateursDto_items_items_employeur.text,declaration_activCollaborateursDto_items_items_descriptionActivite.text,declaration_participationDirigeantDto_items_items_remuneration.montant.montant_annee.text,declaration_participationDirigeantDto_items_items_remuneration.montant.montant_montant.text
0,27/11/2022 18:18:23,fa8d18ec-0db9-4a39-b1f4-caba0c31329b,ADEL,true,VUE_PDF_DU_RECEPISSE_DU_DEPOT_XML,20171221,true,CREATION,Fonction exercée du 20 Mai au 4 Juillet 2022 \...,"Ministre des Solidarités, de l'Autonomie et de...",...,877,néant,83,Non,CREATION,LOPES MAGNUSON,Néant,NaN,2015,0
1,27/11/2022 18:18:23,fa8d18ec-0db9-4a39-b1f4-caba0c31329b,ADEL,true,VUE_PDF_DU_RECEPISSE_DU_DEPOT_XML,20171221,true,CREATION,Fonction exercée du 20 Mai au 4 Juillet 2022 \...,"Ministre des Solidarités, de l'Autonomie et de...",...,877,néant,83,Non,CREATION,LOPES MAGNUSON,Néant,NaN,2016,0
2,27/11/2022 18:18:23,fa8d18ec-0db9-4a39-b1f4-caba0c31329b,ADEL,true,VUE_PDF_DU_RECEPISSE_DU_DEPOT_XML,20171221,true,CREATION,Fonction exercée du 20 Mai au 4 Juillet 2022 \...,"Ministre des Solidarités, de l'Autonomie et de...",...,877,néant,83,Non,CREATION,LOPES MAGNUSON,Néant,NaN,2017,0
3,27/11/2022 18:18:23,fa8d18ec-0db9-4a39-b1f4-caba0c31329b,ADEL,true,VUE_PDF_DU_RECEPISSE_DU_DEPOT_XML,20171221,true,CREATION,Fonction exercée du 20 Mai au 4 Juillet 2022 \...,"Ministre des Solidarités, de l'Autonomie et de...",...,877,néant,83,Non,CREATION,FOUGNIES REBECCA,Néant,NaN,2015,0
4,27/11/2022 18:18:23,fa8d18ec-0db9-4a39-b1f4-caba0c31329b,ADEL,true,VUE_PDF_DU_RECEPISSE_DU_DEPOT_XML,20171221,true,CREATION,Fonction exercée du 20 Mai au 4 Juillet 2022 \...,"Ministre des Solidarités, de l'Autonomie et de...",...,877,néant,83,Non,CREATION,FOUGNIES REBECCA,Néant,NaN,2016,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8795,27/11/2022 18:18:23,fa8d18ec-0db9-4a39-b1f4-caba0c31329b,ADEL,true,VUE_PDF_DU_RECEPISSE_DU_DEPOT_XML,20171221,true,CREATION,Fonction exercée du 20 Mai au 4 Juillet 2022 \...,"Ministre des Solidarités, de l'Autonomie et de...",...,6552,NEANT,20,Non,CREATION,ASTIC ANTOINE,INIUM SAS,Conseil en financement de l'innovation,2018,0
8796,27/11/2022 18:18:23,fa8d18ec-0db9-4a39-b1f4-caba0c31329b,ADEL,true,VUE_PDF_DU_RECEPISSE_DU_DEPOT_XML,20171221,true,CREATION,Fonction exercée du 20 Mai au 4 Juillet 2022 \...,"Ministre des Solidarités, de l'Autonomie et de...",...,6552,NEANT,20,Non,CREATION,ASTIC ANTOINE,INIUM SAS,Conseil en financement de l'innovation,2019,0
8797,27/11/2022 18:18:23,fa8d18ec-0db9-4a39-b1f4-caba0c31329b,ADEL,true,VUE_PDF_DU_RECEPISSE_DU_DEPOT_XML,20171221,true,CREATION,Fonction exercée du 20 Mai au 4 Juillet 2022 \...,"Ministre des Solidarités, de l'Autonomie et de...",...,6552,NEANT,20,Non,CREATION,ASTIC ANTOINE,INIUM SAS,Conseil en financement de l'innovation,2020,0
8798,27/11/2022 18:18:23,fa8d18ec-0db9-4a39-b1f4-caba0c31329b,ADEL,true,VUE_PDF_DU_RECEPISSE_DU_DEPOT_XML,20171221,true,CREATION,Fonction exercée du 20 Mai au 4 Juillet 2022 \...,"Ministre des Solidarités, de l'Autonomie et de...",...,6552,NEANT,20,Non,CREATION,ASTIC ANTOINE,INIUM SAS,Conseil en financement de l'innovation,

# exploring the new dataset

In [115]:
exploded_df.describe().T

,count,unique,top,freq
declaration_dateDepot_text,8800,1,27/11/2022 18:18:23,8800
declaration_uuid_text,8800,1,fa8d18ec-0db9-4a39-b1f4-caba0c31329b,8800
declaration_origine_text,8800,1,ADEL,8800
declaration_complete_text,8800,1,true,8800
declaration_attachedFiles_attachedFiles_fileName_text,8800,1,VUE_PDF_DU_RECEPISSE_DU_DEPOT_XML,8800
...,...,...,...,...
declaration_activCollaborateursDto_items_items_nom.text,8800,4,LOPES MAGNUSON,2200
declaration_activCollaborateursDto_items_items_employeur.text,8800,2,Néant,6600
declaration_activCollaborateursDto_items_items_descriptionActivite.text,2200,1,Conseil en financement de l'innovation,2200
declaration_participationDirigeantDto_items_items_remuneration.montant.montant_annee.text,8800,8,2019,1600


## replace weird char in salaries and convert to int

In [116]:
exploded_df['declaration_mandatElectifDto_items_items_remuneration.montant.montant_montant.text'] = exploded_df['declaration_mandatElectifDto_items_items_remuneration.montant.montant_montant.text'].str.replace(' ', '').astype(int)

In [117]:
exploded_df['declaration_mandatElectifDto_items_items_remuneration.montant.montant_montant.text'].value_counts()

0        3520
16386     704
67047     352
71042     352
71105     352
70773     352
70676     352
27289     352
16865     352
23035     352
20120     352
28007     352
24201     352
16384     352
6827      352
Name: declaration_mandatElectifDto_items_items_remuneration.montant.montant_montant.text, dtype: int64

## cleaning a small sample

In [118]:
def select_colname_like(exploded_df, colname):
  columns_to_select = [col for col in exploded_df.columns if colname in col]
  
  # add current date
  columns_to_select.append("declaration_dateDepot_text")

  df_selected = exploded_df[columns_to_select]

  # Rename column
  new_column_name = colname + "_declaration_dateDepot_text"
  df_selected.rename(columns={"declaration_dateDepot_text": new_column_name}, inplace=True)



  # we have enough columns to act as a key
  return df_selected.drop_duplicates()


# select all columns that contain the string "declaration_mandatElectifDto"
colname = 'declaration_mandatElectifDto'
df_selected = select_colname_like(exploded_df, colname)
df_selected

<ipython-input-118-635a0ab64f80>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected.rename(columns={"declaration_dateDepot_text": new_column_name}, inplace=True)


,declaration_mandatElectifDto_neant_text,declaration_mandatElectifDto_items_items_motif.id.text,declaration_mandatElectifDto_items_items_commentaire.text,declaration_mandatElectifDto_items_items_descriptionMandat.text,declaration_mandatElectifDto_items_items_remuneration.brutNet.text,declaration_mandatElectifDto_items_items_dateDebut.text,declaration_mandatElectifDto_items_items_dateFin.text,declaration_mandatElectifDto_items_items_remuneration.montant.montant_annee.text,declaration_mandatElectifDto_items_items_remuneration.montant.montant_montant.text,declaration_mandatElectifDto_declaration_dateDepot_text
0,false,CREATION,REVENUS NETS IMPOSABLES\n [Données non ...,DEPUTE,Net,01/2017,NaN,2017,67047,27/11/2022 18:18:23
352,false,CREATION,REVENUS NETS IMPOSABLES\n [Données non ...,DEPUTE,Net,01/2017,NaN,2018,71042,27/11/2022 18:18:23
704,false,CREATION,REVENUS NETS IMPOSABLES\n [Données non ...,DEPUTE,Net,01/2017,NaN,2019,71105,27/11/2022 18:18:23
1056,false,CREATION,REVENUS NETS IMPOSABLES\n [Données non ...,DEPUTE,Net,01/2017,NaN,2020,70773,27/11/2022 18:18:23
1408,false,CREATION,REVENUS NETS IMPOSABLES\n [Données non ...,DEPUTE,Net,01/2017,NaN,2021,70676,27/11/2022 18:18:23
1760,false,CREATION,REVENUS NETS IMPOSABLES\n [Données non ...,DEPUTE,Net,01/2017,NaN,2022,27289,27/11/2022 18:18:23
2112,false,CREATION,REVENUS NETS IMPOSABLES\n [Données non ...,PRESIDENT DU DEPARTEMENT 01,Net,04/2015,07/2017,2015,16865,27/11/2022 18:18:23
2464,false,CREATION,REVENUS NETS IMPOSABLES\n [Données non ...,PRESIDENT DU DEPARTEMENT 01,Net,04/2015,07/2017,2016,23035,27/11/2022 18:18:23
2816,false,CREATION,REVENUS NETS IMPOSABLES\n [Données non ...,PRESIDENT DU DEPARTEMENT 01,Net,04/2015,07/2017,2017,20120,27/11/2022 18:18:23
3168,false,CREATION,J'ai été élu président de groupe LR le 6 Novem...,Président du Groupe Les Républicains à l'Assem...,Net,11/2019,05/2022,2019,0,27/11/2022 18:18:23


In [119]:
import os

def clean_df_colnames(df_cleaned):
  
  def remove_common_prefix(strings):
      prefix = os.path.commonprefix(strings)
      if prefix == '':
          return strings
      else:
          return [string[len(prefix):] for string in strings]

  def remove_common_suffix(strings):
      suffix = os.path.commonprefix([string[::-1] for string in strings])[::-1]
      if suffix == '':
          return strings
      else:
          return [string[:-len(suffix)].rstrip('. ') if string.endswith(suffix) else string for string in strings]



  # Assuming you have a DataFrame called 'df' with the long column names
  column_names = df_cleaned.columns.tolist()

  # Remove common prefix from column names
  new_column_names = remove_common_prefix(column_names)
  new_column_names = remove_common_suffix(new_column_names)

  # remove frequent sequence identifier
  new_column_names = [colname.replace('items_items_', '') for colname in new_column_names]


  # Assign new column names to the DataFrame
  df_cleaned.columns = new_column_names
  return df_cleaned

df_cleaned = df_selected.copy()
df_cleaned = clean_df_colnames(df_cleaned)
df_cleaned

,neant_,motif.id,commentaire,descriptionMandat,remuneration.brutNet,dateDebut,dateFin,remuneration.montant.montant_annee,remuneration.montant.montant_montant,declaration_dateDepot_
0,false,CREATION,REVENUS NETS IMPOSABLES\n [Données non ...,DEPUTE,Net,01/2017,NaN,2017,67047,27/11/2022 18:18:23
352,false,CREATION,REVENUS NETS IMPOSABLES\n [Données non ...,DEPUTE,Net,01/2017,NaN,2018,71042,27/11/2022 18:18:23
704,false,CREATION,REVENUS NETS IMPOSABLES\n [Données non ...,DEPUTE,Net,01/2017,NaN,2019,71105,27/11/2022 18:18:23
1056,false,CREATION,REVENUS NETS IMPOSABLES\n [Données non ...,DEPUTE,Net,01/2017,NaN,2020,70773,27/11/2022 18:18:23
1408,false,CREATION,REVENUS NETS IMPOSABLES\n [Données non ...,DEPUTE,Net,01/2017,NaN,2021,70676,27/11/2022 18:18:23
1760,false,CREATION,REVENUS NETS IMPOSABLES\n [Données non ...,DEPUTE,Net,01/2017,NaN,2022,27289,27/11/2022 18:18:23
2112,false,CREATION,REVENUS NETS IMPOSABLES\n [Données non ...,PRESIDENT DU DEPARTEMENT 01,Net,04/2015,07/2017,2015,16865,27/11/2022 18:18:23
2464,false,CREATION,REVENUS NETS IMPOSABLES\n [Données non ...,PRESIDENT DU DEPARTEMENT 01,Net,04/2015,07/2017,2016,23035,27/11/2022 18:18:23
2816,false,CREATION,REVENUS NETS IMPOSABLES\n [Données non ...,PRESIDENT DU DEPARTEMENT 01,Net,04/2015,07/2017,2017,20120,27/11/2022 18:18:23
3168,false,CREATION,J'ai été élu président de groupe LR le 6 Novem...,Président du Groupe Les Républicains à l'Assem...,Net,11/2019,05/2022,2019,0,27/11/2022 18:18:23


### 

In [120]:
# select all columns that contain the string "declaration_mandatElectifDto"
colname = 'declaration_participationFinanciereDto'
df_selected = select_colname_like(exploded_df, colname)
df_selected

<ipython-input-118-635a0ab64f80>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected.rename(columns={"declaration_dateDepot_text": new_column_name}, inplace=True)


,declaration_participationFinanciereDto_neant_text,declaration_participationFinanciereDto_items_items_motif.id.text,declaration_participationFinanciereDto_items_items_commentaire.text,declaration_participationFinanciereDto_items_items_nomSociete.text,declaration_participationFinanciereDto_items_items_evaluation.text,declaration_participationFinanciereDto_items_items_remuneration.text,declaration_participationFinanciereDto_items_items_nombreParts.text,declaration_participationFinanciereDto_items_items_actiConseil.text,declaration_participationFinanciereDto_declaration_dateDepot_text
0,false,CREATION,[Données non publiées],ORANGE,877,néant,83,Non,27/11/2022 18:18:23
12,false,CREATION,[Données non publiées],CREDIT AGRICOLE SA,2910,néant,341,Non,27/11/2022 18:18:23
24,false,CREATION,[Données non publiées],AIRBUS,1929,NEANT,20,Non,27/11/2022 18:18:23
36,false,CREATION,[Données non publiées],L'OREAL,6552,NEANT,20,Non,27/11/2022 18:18:23


In [121]:
df_cleaned = df_selected.copy()
df_cleaned = clean_df_colnames(df_cleaned)
df_cleaned

,neant_,motif.id,commentaire,nomSociete,evaluation,remuneration,nombreParts,actiConseil,declaration_dateDepot_
0,false,CREATION,[Données non publiées],ORANGE,877,néant,83,Non,27/11/2022 18:18:23
12,false,CREATION,[Données non publiées],CREDIT AGRICOLE SA,2910,néant,341,Non,27/11/2022 18:18:23
24,false,CREATION,[Données non publiées],AIRBUS,1929,NEANT,20,Non,27/11/2022 18:18:23
36,false,CREATION,[Données non publiées],L'OREAL,6552,NEANT,20,Non,27/11/2022 18:18:23
